In [ ]:
# SPDX-License-Identifier: BSD-3-Clause
# Copyright 2022 NXP

# Models download
Purpose of this notebook is to download model sources and metadata used by the examples, and apply necessary conversion steps prior to execution. Resulting models will be located in the top level `downloads` directory.

This notebook has dependency on availability of eIQ Toolkit environment being configured.
Refer to [downloads\README.md](./README.md) for instructions.

## Preamble


In [ ]:
import os
from pathlib import Path
import re
import shutil
import subprocess
import tarfile
import urllib.parse
import urllib.request
import zipfile

cwd = Path.cwd()

# Repository directory
top_dir = str(cwd.parent.absolute())
# Models directory
models_dir = os.path.join(top_dir, 'downloads', 'models')
# Media directory
media_dir = os.path.join(top_dir, 'downloads', 'media')
# Work directory (temporary)
tmp_dir = os.path.join(top_dir, 'tmp')
# Small batch of pictures samples for post-training quantization
samples_dir = os.path.join(top_dir, 'samples')

# notebook dependency check:
# eIQ environment verification
if shutil.which('deepview-converter') is None:
    raise FileNotFoundError('Need eIQ Toolkit environment setup - see downloads/README.md')

# 
# General purpose helpers
#

# Download a list of files from urls
# returns list of local file paths
def fetch_urls(urls, dest_dir):
    Path(dest_dir).mkdir(parents=True, exist_ok=True)
    files = []
    for url in urls:
        path = urllib.parse.urlparse(url).path
        file = os.path.basename(path)
        dest_file = os.path.join(dest_dir, file)
        print(f"downloading {url} ({dest_file})")
        # wikimedia.org requires user agent
        req = urllib.request.Request(
                  url, data=None,
                  headers= {'User-Agent' : 'download script/0.1'}
              )
        with urllib.request.urlopen(req) as resp:
            with open(dest_file, 'wb') as dest:
                content = resp.read()
                dest.write(content)
                files += [ dest_file ]
    return files

# download files and tarballs - extract archives when relevant
def fetcher(items):
    for item in items:
        url = item['url']
        dest = item['dest']
        download = fetch_urls([url], dest)[0]
        if download.endswith('.zip'):
            with zipfile.ZipFile(download) as _zip:
                print(f"extracting zip {download}")
                _zip.extractall(path=dest)
        if download.endswith('.tar.gz') or download.endswith('.tgz'):
            with tarfile.open(download) as _tar:
                print(f"extracting tar {download}")
                _tar.extractall(path=dest)

# Create list of clean (empty) directories
def make_clean_dirs(dest_dirs):
    for dest_dir in dest_dirs:
        shutil.rmtree(dest_dir, ignore_errors=True)
        Path(dest_dir).mkdir(parents=True)

# Invoke deepview-converter with args
def dvc(origin, dest, args=[]):
    cmd = ['deepview-converter'] + args + [origin, dest]
    subprocess.run(cmd, check=True)

# Convert TFLite model (.tflite) to DeepViewRT (.rtm)
def dvc_tflite_rtm(origin):
    dest = re.sub('.tflite', '.rtm', origin)
    dvc(origin, dest)
    
# Convert TFLite model (<name>.tflite) to Vela-processed TFLite (<name>_vela.tflite)
def vela(origin, args=[]):
    cmd = ['vela']
    cmd += ['--output-dir', os.path.dirname(origin)]
    cmd += args
    cmd += [origin]
    subprocess.run(cmd, check=True)


In [ ]:
# Download a few jpeg picture samples for post-training quantization (for demo)
shutil.rmtree(samples_dir, ignore_errors=True)    
samples = [
    'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2d/Olea_europaea_cuspidata-africana_Cape_Town.JPG/605px-Olea_europaea_cuspidata-africana_Cape_Town.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Sport-orange-red-color-tennis-organ-491989-pxhere.jpg/640px-Sport-orange-red-color-tennis-organ-491989-pxhere.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8e/Domesticated_goose_head%2C_Chaguaramal%2C_Venezuela.jpg/640px-Domesticated_goose_head%2C_Chaguaramal%2C_Venezuela.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/20080831-R0012506.JPG/640px-20080831-R0012506.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Pisa_Cathedral_%26_Bell_Tower_-_%22Leaning_Tower_of_Pisa%22_%289809628764%29.jpg/640px-Pisa_Cathedral_%26_Bell_Tower_-_%22Leaning_Tower_of_Pisa%22_%289809628764%29.jpg',
    ]
fetch_urls(samples, samples_dir)

# Detection
## Artifacts download
model: [ssdlite_mobilenet_v2_coco](ssdlite_mobilenet_v2_coco)<br>
reference: [TensorFlow 1 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md#mobile-models)

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'detection')
make_clean_dirs([tmp_dir, dest_dir])

# models and metadata to be downloaded from network
model_float = 'ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz'
url_float = urllib.parse.urljoin('http://download.tensorflow.org/models/object_detection/', model_float)
packages = [
    {'url'  : url_float,
     'dest' : tmp_dir},
    {'url'  : 'https://github.com/nnsuite/testcases/raw/master/DeepLearningModels/tensorflow-lite/ssd_mobilenet_v2_coco/coco_labels_list.txt',
     'dest' : dest_dir},
    {'url'  : 'https://github.com/nnsuite/testcases/raw/master/DeepLearningModels/tensorflow-lite/ssd_mobilenet_v2_coco/box_priors.txt',
     'dest' : dest_dir},
]

fetcher(packages)

folder = re.sub('.tar.gz', '', model_float)
pb_model = os.path.join(tmp_dir, folder, 'frozen_inference_graph.pb')

model_darknet = 'yolov4-tiny.weights'
url_darknet = urllib.parse.urljoin('https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights', model_darknet)
packages = [
    {'url'  : url_darknet,
     'dest' : tmp_dir},
    {'url'  : 'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-2014_2017.txt',
     'dest' : dest_dir},
]

fetcher(packages)

## Models conversions

In [ ]:
#
# TFLite models (without box-decoding / NMS postprocessing)
#

# convert TF -> TFLite (float)
tflite_float = os.path.join(dest_dir, 'ssdlite_mobilenet_v2_coco_no_postprocess.tflite')
args =  ['--tflite_converter', 'toco', '--default_shape', '1,300,300,3']
args += ['--input_names', 'Preprocessor/sub', '--output_names', 'concat,concat_1']
dvc(pb_model, tflite_float, args)

# convert TF -> TFLite (quantized)
tflite_quant = os.path.join(dest_dir, 'ssdlite_mobilenet_v2_coco_quant_uint8_float32_no_postprocess.tflite')
args  = ['--tflite_converter', 'toco', '--default_shape', '1,300,300,3']
args += ['--quantize', '--quantize_format', 'int8']
args += ['--input-type', 'uint8', '--output-type', 'float32']
args += ['--input_names', 'Preprocessor/sub', '--output_names', 'concat,concat_1']
dvc(pb_model, tflite_quant, args)

In [ ]:
# Convert Yolov4-tiny Darknet -> TFlite (quantized)
# this script may take several minutes to complete
!python3 {top_dir}/detection/yolov4-tiny_export_model.py \
--weights_path={tmp_dir} --output_path={dest_dir} --images_path={samples_dir}

In [ ]:
#
# DeepView RT models - converted from TFLite
#

# convert TFLite (float) -> DeepViewRT (float)
dvc_tflite_rtm(tflite_float)

# convert TFLite (quantized) -> DeepViewRT (quantized)
dvc_tflite_rtm(tflite_quant)


In [ ]:
#
# Vela-processed TFLite models targeting i.MX 93 Ethos-U NPU - converted from TFLite
#

# convert TFLite (quantized) -> Vela TFLite (quantized)
vela(tflite_quant)

In [ ]:
# Process Vela on quantized Yolov4-tiny

# The model must have been exported from Darknet to TFLite before
yolov4_tiny_416_quant = os.path.join(dest_dir, 'yolov4-tiny_416_quant.tflite')

# convert TFLite (quantized) -> Vela TFLite (quantized)
vela(yolov4_tiny_416_quant)

#  Classification
## Artifacts download
model: [mobilenet_v1_1.0_224](http://download.tensorflow.org/models/mobilenet_v1_2018_02_22/mobilenet_v1_1.0_224.tgz)<br>
reference: [TensorFlow-Slim image classification model library](https://github.com/tensorflow/models/tree/master/research/slim)

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'classification')
make_clean_dirs([tmp_dir, dest_dir])

# models and metadata to be downloaded from network
model_float = 'mobilenet_v1_1.0_224.tgz'
model_quant = 'mobilenet_v1_1.0_224_quant.tgz'
url_model = 'http://download.tensorflow.org/models/mobilenet_v1_2018_08_02/'
packages = [
    {'url'  : urllib.parse.urljoin(url_model, model_float),
     'dest' : tmp_dir},
    {'url'  : urllib.parse.urljoin(url_model, model_quant),
     'dest' : tmp_dir},
    {'url'  : 'https://storage.googleapis.com/download.tensorflow.org/models/tflite/mobilenet_v1_1.0_224_quant_and_labels.zip',
     'dest' : tmp_dir},
]

fetcher(packages)

pb_model_float = os.path.join(tmp_dir, 'mobilenet_v1_1.0_224_frozen.pb')
pb_model_quant = os.path.join(tmp_dir, 'mobilenet_v1_1.0_224_quant_frozen.pb')

labels_file = os.path.join(tmp_dir, 'labels_mobilenet_quant_v1_224.txt')
shutil.copy2(labels_file, dest_dir)


## Models conversions

In [ ]:
#
# TFLite models
#

# convert TF -> TFLite (float)
tflite_float = os.path.join(dest_dir, 'mobilenet_v1_1.0_224.tflite')
args =  ['--tflite_converter', 'toco', '--default_shape', '1,224,224,3']
args += ['--input_names', 'input', '--output_names', 'MobilenetV1/Predictions/Reshape_1']
dvc(pb_model_float, tflite_float, args)

# convert TF -> TFLite (quantized)
tflite_quant = os.path.join(dest_dir, 'mobilenet_v1_1.0_224_quant_uint8_float32.tflite')
args =  ['--tflite_converter', 'toco', '--default_shape', '1,224,224,3']
args += ['--quantize', '--quantize_format', 'int8']
args += ['--input-type', 'uint8', '--output-type', 'float32']
args += ['--samples', samples_dir]
args += ['--input_names', 'input', '--output_names', 'MobilenetV1/Predictions/Reshape_1']
dvc(pb_model_float, tflite_quant, args)


In [ ]:
#
# DeepView RT models - converted from TFLite
#

# convert TFLite (float) -> DeepViewRT (float)
dvc_tflite_rtm(tflite_float)

# convert TFLite (quantized) -> DeepViewRT (quantized)
dvc_tflite_rtm(tflite_quant)


In [ ]:
#
# Vela-processed TFLite models targeting i.MX 93 Ethos-U NPU - converted from TFLite
#

# convert TFLite (quantized) -> Vela TFLite (quantized)
vela(tflite_quant)

# Segmentation
## Artifacts download
model: [deeplabv3_mnv2_dm05](http://download.tensorflow.org/models/deeplabv3_mnv2_dm05_pascal_trainaug_2018_10_01.tar.gz)<br>
references: 
1. [TensorFlow DeepLab Model Zoo](https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/model_zoo.md)<br>
2. [Quantize DeepLab model for faster on-device inference](https://github.com/tensorflow/models/blob/master/research/deeplab/g3doc/quantize.md)

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'segmentation')
images_dir = os.path.join(media_dir, 'pascal_voc_2012_images')
make_clean_dirs([tmp_dir, dest_dir, images_dir])

# models and metadata to be downloaded from network
model_float = 'deeplabv3_mnv2_dm05_pascal_trainaug_2018_10_01.tar.gz'
model_quant = 'deeplabv3_mnv2_dm05_pascal_train_aug_8bit_2019_04_26.tar.gz'
url_model = 'http://download.tensorflow.org/models/'
packages = [
    {'url'  : urllib.parse.urljoin(url_model, model_float),
     'dest' : tmp_dir},
    {'url'  : urllib.parse.urljoin(url_model, model_quant),
     'dest' : tmp_dir},
]

fetcher(packages)

pb_model_float = os.path.join(tmp_dir, 'deeplabv3_mnv2_dm05_pascal_trainaug','frozen_inference_graph.pb')
pb_model_quant = os.path.join(tmp_dir, 'deeplabv3_mnv2_dm05_pascal_trainaug_8bit','frozen_inference_graph.pb')

# fetch and rename a few images relevant to pascal voc 2012 data set
images = [
    'https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/Plectranthus_verticillatus_in_a_ceramic_pot.jpg/640px-Plectranthus_verticillatus_in_a_ceramic_pot.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f2/Maceta.jpg/566px-Maceta.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/f/f6/Stray_Cat%2C_Nafplio.jpg/800px-Stray_Cat%2C_Nafplio.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/b/bd/Feral_cat_7.jpg/800px-Feral_cat_7.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/0/0a/Senior-lady-dog-walker.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Texel_ewe_and_three_lambs.jpg/800px-Texel_ewe_and_three_lambs.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Cow_farm_in_Bemmel%2C_Lingewaard.jpg/800px-Cow_farm_in_Bemmel%2C_Lingewaard.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/6/62/Urban_cycling_III.jpg/800px-Urban_cycling_III.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/2/2e/LomondRoadsTTRider.jpg/467px-LomondRoadsTTRider.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Interceptor_of_Bangalore_Traffic_Police.jpg/640px-Interceptor_of_Bangalore_Traffic_Police.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c0/Ranger_clearing_traffic_at_a_bison_jam_in_Lamar_Valley_%2848fe8fe3-c1e3-4828-acec-496eaef43503%29.jpg/800px-Ranger_clearing_traffic_at_a_bison_jam_in_Lamar_Valley_%2848fe8fe3-c1e3-4828-acec-496eaef43503%29.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c7/Finnair.a320-200.oh-lxf.arp.jpg/800px-Finnair.a320-200.oh-lxf.arp.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ef/G-BGMP_Reims_F172_%40Cotswold_Airport%2C_July_2005.jpg/800px-G-BGMP_Reims_F172_%40Cotswold_Airport%2C_July_2005.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/JNR_14_series_sleeper_Blue_Train_Hayabusa.jpg/640px-JNR_14_series_sleeper_Blue_Train_Hayabusa.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/7122_series_train_%2812%29.JPG/640px-7122_series_train_%2812%29.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Blue_fishing_boat_harbour_Eretria_Euboea_Greece.jpg/611px-Blue_fishing_boat_harbour_Eretria_Euboea_Greece.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/KYOEI_boat_in_Kisarazu_port_3.jpg/640px-KYOEI_boat_in_Kisarazu_port_3.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Jarritos_glass_bottle_%28Mexico%29.jpg/450px-Jarritos_glass_bottle_%28Mexico%29.jpg',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Andechser_3_beers.JPG/798px-Andechser_3_beers.JPG',
    'https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/Ski_Citrus_Soda_%2825395355114%29.jpg/640px-Ski_Citrus_Soda_%2825395355114%29.jpg',
]
files = fetch_urls(images, tmp_dir)
count = 0
# rename files name as <image%04d.jpg>
for src in files:
    dest = os.path.join(images_dir, f'image{count:04}.jpg')
    count += 1
    shutil.copy2(src, dest)


## Models conversions

In [ ]:
#
# TFLite models
#

# convert TF -> TFLite (float)
tflite_float = os.path.join(dest_dir, 'deeplabv3_mnv2_dm05_pascal.tflite')
args = ['--tflite_converter', 'toco', '--default_shape', '1,513,513,3']
args += ['--input_names', 'MobilenetV2/MobilenetV2/input', '--output_names', 'ResizeBilinear_2']
dvc(pb_model_float, tflite_float, args)

# convert TF -> TFLite (quantized)
tflite_quant = os.path.join(dest_dir, 'deeplabv3_mnv2_dm05_pascal_quant_uint8_float32.tflite')
args =  ['--tflite_converter', 'toco', '--default_shape', '1,513,513,3']
args += ['--quantize', '--quantize_format', 'int8']
args += ['--input-type', 'uint8', '--output-type', 'float32']
args += ['--samples', samples_dir]
args += ['--input_names', 'MobilenetV2/MobilenetV2/input', '--output_names', 'ResizeBilinear_2']
dvc(pb_model_float, tflite_quant, args)


# Pose
## Artifacts download
model: [movenet/singlepose/lightning TF2 SavedModel](https://storage.googleapis.com/tfhub-modules/google/movenet/singlepose/lightning/4.tar.gz)<br>
references: 
1. [TensorFlow Hub Image Pose Detection](https://tfhub.dev/s?module-type=image-pose-detection)<br>
2. [TensorFlow Pose](https://www.tensorflow.org/lite/examples/pose_estimation/overview)

In [ ]:
# start from clean state
dest_dir = os.path.join(models_dir, 'pose')
make_clean_dirs([tmp_dir, dest_dir])
movies_dir = os.path.join(media_dir, 'movies')

# models and metadata to be downloaded from network
url_model = 'https://storage.googleapis.com/tfhub-modules/google/movenet/singlepose/lightning/4.tar.gz'
packages = [{'url' :url_model, 'dest': tmp_dir},]
fetcher(packages)

pb_model_float = os.path.join(tmp_dir, 'saved_model.pb')

# Download movie
url_movie = 'https://upload.wikimedia.org/wikipedia/commons/transcoded/1/17/Conditioning_Drill_1-_Power_Jump.webm/Conditioning_Drill_1-_Power_Jump.webm.480p.vp9.webm'
packages = [{'url': url_movie, 'dest': movies_dir},]
fetcher(packages)


## Models conversions

In [ ]:
#
# TFLite models
#

# convert TF -> TFLite (float)
# XXX: Toco converter crashes
tflite_float = os.path.join(dest_dir, 'movenet_single_pose_lightning.tflite')
args=[]
dvc(pb_model_float, tflite_float, args)

# TODO: quantized model


# Postamble / cleanup

In [ ]:
# Remove temporary directories
shutil.rmtree(tmp_dir, ignore_errors=True)
shutil.rmtree(samples_dir, ignore_errors=True)